Importar librerías

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.api import OLS

Importar datos

In [3]:
d1 = pd.read_csv('../dat/baseline.csv')
d2 = pd.read_csv('../dat/endline.csv')
d3 = pd.read_csv('../dat/completa.csv')